Übung: Dedizierte Docker Image-Stände bauen und verwalten
----------------------------------------------------------------------------------------

Nun wollen wir ein Docker Image erstellen, dazu brauchen wir ein Dockerfile:

In [ ]:
%%bash
cat <<%EOF% >Dockerfile
#
#   Einfache MySQL Umgebung - veraltet nur zu Schulungszwecken!
#
FROM ubuntu:14.04

# root Password setzen, damit kein Dialog erscheint und die Installation haengt!
# Security Problem, besser: https://12factor.net/de/config
RUN echo 'mysql-server mysql-server/root_password password S3cr3tp4ssw0rd' | debconf-set-selections 
RUN echo 'mysql-server mysql-server/root_password_again password S3cr3tp4ssw0rd' | debconf-set-selections 

# Installation
RUN apt-get update && apt-get install -y mysql-server

# mysql config - Port fuer alle Hosts oeffnen 
# Wiederspricht Linux Logik alle Ports zu schliessen. Geht aber nicht anders weil jeder Container seine eigene IP-Adresse hat.
RUN sed -i -e"s/^bind-address\s*=\s*127.0.0.1/bind-address = 0.0.0.0/" /etc/mysql/my.cnf

# Obsolet, siehe Kubernetes Health Probe Pattern
HEALTHCHECK --interval=5m --timeout=3s CMD curl -f localhost:3306 || exit 1

# Obsolet, siehe Kubernetes Ressource: Service
EXPOSE 3306

# Obsolet, siehe Kubernetes PersistenVolumeClaims
VOLUME /var/lib/mysql

CMD ["mysqld"]
%EOF%

Um das Anschliessend das Docker Image zu bilden

In [ ]:
! docker build -t mysql .

Das Ergebnis können wir uns mit `docker image ls` anschauen

In [ ]:
! docker image ls

Ebenfalls können die einzelnen Buildschritte sichtbar gemacht werden

In [ ]:
! docker history mysql

***
Fragen
======

Beantwortet die Fragen einzeln oder in der Gruppe und vergleicht diese mit den Antworten


Wie erstelle ich ein Container Image
<details><summary>Antwort</summary>
    docker build 
</p></details>

---

In welcher Datei steht welche Inhalte sich im Container Image befinden?
<details><summary>Antwort</summary>
    Dockerfile 
</p></details>

---

Welche Aussage ist besser (siehe auch [The Twelve-Factor App](https://12factor.net/))?
* a) Dockerfile sollten möglichst das Builden (CI) und Ausführen von Services beinhalten, so ist alles an einem Ort und der Entwickler kann alles erledigen.
* b) Das Builden und Ausführen von Services ist strikt zu trennen. Damit saubere und nachvollziehbare Services mittels CI/CD Prozess entstehen. 
<details><summary>Antwort</summary>
    b)
</p></details>

---

Warum sollten Versionen `tag` von Images immer angegeben werden?
<details><summary>Antwort</summary>
	 	Ansonsten wird `latest` verwendet und so nicht sicher welche Version wirklich verwendet wird.
</p></details>

---

Was ist der Unterschied zwischen `docker save`/`docker load` und `docker export`/`docker import`?
<details><summary>Antwort</summary>
        save/load ist für Images, export/import für Container.
        So ist es möglich auch ohne Docker Registry Container Images auszutauschen, z.B. in einer Bank.
</p></details>

---